# L6: Check outputs

## Configuration
#### Chargement de la clé OpenAI et des librairies Python

In [10]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)


In [11]:
def get_completion_from_messages(messages, 
                                 model="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

### Vérifiez la sortie pour détecter tout contenu potentiellement nuisible

In [16]:
final_response_to_customer = f"""
Le SmartX ProPhone a un écran de 6,1 pouces, 128 Go de stockage, 
un double appareil photo de 12 MP et 5G. L'appareil photo reflex 
numérique FotoSnap a un capteur de 24,2 MP, une vidéo en 1080p, 
un écran LCD de 3 pouces et des objectifs interchangeables.
Nous avons une variété de téléviseurs, y compris le CineView 4K TV 
avec un écran de 55 pouces, une résolution 4K, du HDR et des fonctionnalités 
de smart TV. Nous avons également le système de cinéma maison SoundMax avec 
5,1 canaux, une puissance de 1000 W, un caisson de basses sans fil et Bluetooth.
Avez-vous des questions spécifiques sur ces produits ou sur d'autres produits que nous proposons ?
"""
response = client.moderations.create(
    input=final_response_to_customer
)
moderation_output = response.results[0]
print(moderation_output)

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_applied_input_types=None, category_scores=CategoryScores(harassment=8.775878814049065e-05, harassment_threatening=5.927726306254044e-06, hate=6.237114575924352e-05, hate_threatening=5.0794125854736194e-05, illicit=None, illicit_violent=None, self_harm=5.266005973680876e-05, self_harm_instructions=8.987857427200652e-07, self_harm_intent=5.816901193611557e-06, sexual=0.002583579160273075, sexual_minors=0.0005325612146407366, violence=0.0010029146214947104, violence_graphic=7.938853377709165e-05, self-harm=5.2660059736

### Vérifiez si la sortie est factuellement basée sur les informations produit fournies. 

In [17]:
system_message = f"""
Vous êtes un assistant qui évalue si les réponses des agents de 
service client répondent suffisamment aux questions des clients, 
et qui valide également que tous les faits que l'assistant cite 
des informations produit sont corrects.
Les informations produit et les messages des utilisateurs et des 
agents de service client seront délimités par 3 accents graves, 
c'est-à-dire ```.
Répondez par un caractère Y ou N, sans ponctuation :
Y - si la sortie répond suffisamment à la question ET que la réponse 
utilise correctement les informations produit
N - sinon

Affichez une seule lettre.
"""
customer_message = f"""Parlez-moi du SmartX ProPhone et de l'appareil 
photo FotoSnap, le reflex numérique. Parlez-moi aussi de vos téléviseurs."""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

La réponse utilise-t-elle correctement les informations récupérées ?
La réponse répond-elle suffisamment à la question ?

Affichez Y ou N.
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


In [18]:
another_response = "La vie est comme une boite de chocolats"
q_a_pair = f"""
Message de l'utilisateur : ```{customer_message}```
Informations sur le produit : ```{product_information}```
Réponse de l'agent : ```{another_response}```

La réponse utilise-t-elle correctement les informations récupérées ?
La réponse répond-elle suffisamment à la question ?

Affichez Y ou N.
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N
